In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
model = load_model('model.h5')


2025-08-17 00:40:15.291754: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-08-17 00:40:15.291984: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-08-17 00:40:15.292002: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-08-17 00:40:15.292222: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-08-17 00:40:15.292257: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)


In [4]:
input_data = {
    'CreditScore': 650,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 5,
    'Balance': 15000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60000
}

In [5]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = onehot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,Male,40,5,15000,2,1,1,60000


In [7]:
input_df['Gender'] = label_encoder.transform(input_df['Gender'])

In [8]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,1,40,5,15000,2,1,1,60000


In [10]:
input_df = pd.concat([input_df.drop(columns=['Geography']), geo_encoded_df], axis=1)

In [11]:
input_scaled  = scaler.transform(input_df)

In [12]:
prediction = model.predict(input_scaled)
prediction

2025-08-17 00:58:03.442454: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 924ms/step


array([[1.8944595e-18]], dtype=float32)

In [13]:
prediction_prob = prediction[0][0]

In [14]:
prediction_prob

1.8944595e-18

In [15]:
if prediction_prob > 0.5:
    print('The customer is likely to chrn')
else: 
    print('The customer is unlikely to chrn')

The customer is unlikely to chrn
